# 簡易版転移学習の例
- 関連記事
    - [テキストデータに対する特徴量設計3（転移学習）](./nlp3.md)
- 本当にやりたいタスクは [The 20 newsgroups text dataset](https://scikit-learn.org/stable/datasets/index.html#the-20-newsgroups-text-dataset) の分類タスク。
- 簡易版転移学習アプローチの例
    - pre-training stage
        - [gensim(fastText)](https://radimrehurek.com/gensim/) で、[swwiki-latest-pages-articles.xml.bz2](https://dumps.wikimedia.org/swwiki/latest/swwiki-latest-pages-articles.xml.bz2) から言語モデルを学習。
    - fine tuning stage
        - 言語モデルを使ってニュース記事をベクトル化。このベクトルを使って分類学習する。
- 比較対象
    - 直接 BoW + TFIDF で分類学習する。他のコーパスを使わず、同じコーパス内での他タスクもしない。

In [1]:
# pre-training stage.
# building dataset from swwiki-latest-pages-articles.xml.bz2, and train a model with fastText
import os, pickle, re, json
import multiprocessing
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.fasttext import FastText as FT_gensim

# 事前学習用のコーパス、学習後のモデルを保存するファイル名の指定
#wikipedia_data = '/Users/tnal/Downloads/data/wikipedia-en/swwiki-latest-pages-articles.xml.bz2'
#path = "/Users/tnal/PycharmProjects/dm_nlp/transfer_learning/gensim"
#model_file = path+'/swwiki'

# 上記のように直接パスやファイル名等を書くのではなく、
# 設定ファイルをまとめた JSON ファイル等を経由して指定すると、
# ソースコード修正をできるだけ抑えつつ、設定箇所を抽出していることによる可用性が高くなる。
config_file = "config_transfer_learning.json"
with open(config_file, "r") as fh:
    config = json.load(fh)

wikipedia_data = config["wikipedia_data"]
path = config["model_path"]
model_file = path + "/" + config["model_base_filename"]

# 時間かかるので、学習済みモデルを上記で指定した場所に保存。
# 既に保存済みのモデルの利用にも対応。
answer = "n"
if os.path.exists(model_file):
    print("# Use model file \"{}\"?".format(model_file))
    answer = input("[y/n] => ")
if answer == "n":
    print("loading data...")
    wiki = WikiCorpus(wikipedia_data, lemmatize=False, dictionary={})
    sentences = list(wiki.get_texts())

    #faxtText
    model = FT_gensim(size=200, window=10, min_count=10, workers=max(1, multiprocessing.cpu_count() - 1))

    # build the vocabulary
    print("building vocab...")
    model.build_vocab(sentences=sentences)

    # train the model
    print("training model...")
    model.train(
        sentences=sentences, epochs=model.epochs,
        total_examples=model.corpus_count, total_words=model.corpus_total_words,
        window=10, min_count=10,
        workers=max(1, multiprocessing.cpu_count() - 1)
    )

    ## Note: 以下のような pickle.dump では不十分。モデル内部全てを保存するには、用意されてるsave関数を使おう。
    #with open(model_file, 'wb') as fh:
    #    pickle.dump(model, fh)
    model.save(model_file) # 3つの関連ファイルが自動生成。合計約3GB。
else:
    model = FT_gensim.load(model_file)

# 動作確認
print(model.wv['artificial'][:5])
print(model.wv["more like funchuck,Gave this"][:5])


# Use model file "/Users/tnal/PycharmProjects/dm_nlp/transfer_learning/gensim/swwiki"?
[y/n] => y
[-0.47669616 -0.30191633  0.36801535 -0.68636864  0.9953687 ]
[-0.06122479 -0.10474044  0.15912144 -0.24299866 -0.24580786]


In [2]:
# fine-tuneing stage.
# デーセットの用意
# こちらも時間かかるので、変換したデータセットを指定した場所に保存。
# 既に保存済みデータセットの利用にも対応。

from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
train_text = newsgroups_train.data
train_label = newsgroups_train.target
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
test_text = newsgroups_test.data
test_label = newsgroups_test.target

# 事前学習したfastTextにより、文章をベクトルに変換
def sentence2vector(sentences, model):
    vectors = []
    for sent in sentences:
        vectors.append(model.wv[sent])
    return vectors

print("convert from text to vector with the model...")
train_file = path+"/"+config["train_file"]
test_file = path+"/"+config["test_file"]

answer = "n"
if os.path.exists(train_file):
    print("# Use train file \"{}\"?".format(train_file))
    answer = input("[y/n] => ")
if answer == "n":
    train_vectors = sentence2vector(train_text, model)
    test_vectors = sentence2vector(test_text, model)
    with open(train_file, "wb") as fh:
        pickle.dump(train_vectors, fh)
    with open(test_file, "wb") as fh:
        pickle.dump(test_vectors, fh)
else:
    with open(train_file, "rb") as fh:
        train_vectors = pickle.load(fh)
    with open(test_file, "rb") as fh:
        test_vectors = pickle.load(fh)

convert from text to vector with the model...
# Use train file "/Users/tnal/PycharmProjects/dm_nlp/transfer_learning/gensim/20news_train.pkl"?
[y/n] => y


In [3]:
# 実際に解決したいタスク（文書分類）を学習するためのモデルを用意し、学習。

from sklearn import svm
print("fine-tuneing...")
clf = svm.SVC(gamma='scale')
clf.fit(train_vectors, train_label)
score = clf.score(test_vectors, test_label)
print(score)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_label, clf.predict(test_vectors))
print(cm)


fine-tuneing...
0.7854137447405329
[[211 108]
 [ 45 349]]


In [4]:
# 比較対象の、事前学習なし実験。
# BoW + TFIDFによるベクトル生成

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
print("train_vectors.shape=", train_vectors.shape)
print("len(train_label)=",len(train_label))

test_vectors = vectorizer.transform(newsgroups_test.data)
print("test_vectors.shape=", test_vectors.shape)
print("len(test_label)=",len(test_label))

# 実際に解決したいタスク（文書分類）を学習するためのモデルを用意し、学習。
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(train_vectors, train_label)
score = clf.score(test_vectors, test_label)
print(score)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_label, clf.predict(test_vectors))
print(cm)


train_vectors.shape= (1073, 22464)
len(train_label)= 1073
test_vectors.shape= (713, 22464)
len(test_label)= 713
0.5525946704067322
[[  0 319]
 [  0 394]]
